# 🤖 Automated Research Paper Analysis System
**Multi-Agent AI System for Academic Research Analysis**

---

**Author**: Mohamed Outahajala  
 
**Framework**: Google Agent Development Kit (ADK)  
---

## Quick Start

1. Upload your PDF research paper as `document.pdf`
2. Set your API keys in `.env`:

## Problem Statement
Analyzing research papers manually is time-consuming and requires:
- Reading lengthy PDFs
- Summarizing key findings
- Researching latest trends
- Synthesizing information from multiple sources

This process can take 2-3 hours per paper.

## Solution
An automated Multi-Agent Research System that:
- Extracts PDF content automatically
- Generates comprehensive summaries
- Performs real-time market research
- Produces structured reports in under 5 minutes

**Value**: Reduces research time by 95%, from 3 hours to 5 minutes per paper.

<!-- if numpy or pandas .. not installed -->
!pip install numpy
!pip install pandas
!pip install pypdf
!pip install google-adk

In [78]:
# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import json

# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Retrieve API keys from environment variables
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_SEARCH_API_KEY = os.getenv("GOOGLE_SEARCH_API_KEY")
MODEL_NAME = os.getenv("MODEL_NAME", "gemini-2.5-flash")  # Default to "gemini-2.5-flash" if not set   



In [79]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [80]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [81]:
# Diagnostic: Check if document.pdf is accessible
import os

print("Current working directory:", os.getcwd())
print("Files in current directory:")
for item in os.listdir('.'):
    if item.endswith('.pdf'):
        print(f"  📄 {item}")

# Check if document.pdf exists
pdf_path = "document.pdf"
if os.path.exists(pdf_path):
    print(f"✅ '{pdf_path}' found and accessible")
    # Quick test read
    from pypdf import PdfReader
    reader = PdfReader(pdf_path)
    print(f"   Pages: {len(reader.pages)}")
    print(f"   First 100 chars: {reader.pages[0].extract_text()[:100]}...")
else:
    print(f"❌ '{pdf_path}' NOT found in current directory")
    print(f"   You may need to specify the full path or move the file")

Current working directory: /Users/admin/HF/Agents5D
Files in current directory:
  📄 document.pdf
✅ 'document.pdf' found and accessible
   Pages: 5
   First 100 chars: Tagging Amazigh with AnCoraPipe 
Mohamed Outahajala, Lahbib Zekouar, Paolo Rosso, M. Antònia Martí 
...


In [82]:
# PDF Search Tool
from pypdf import PdfReader

def search_pdf_tool(file_path: str, query: str) -> str:
    """
    Searches for keywords within a PDF file and returns relevant text snippets.
    If the file is not found, returns mock data for demonstration.
    """
    print(f"    🔎 [Tool] Searching PDF '{file_path}' for: '{query}'")
    
    # 1. Try to read the actual file
    if os.path.exists(file_path):
        try:
            reader = PdfReader(file_path)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
            
            # Return full document text instead of keyword matching
            if text.strip():
                return text[:5000]  # Return first 5000 chars to avoid token limits
            else:
                return "PDF file is empty or unreadable."
        except Exception as e:
            return f"Error reading PDF: {e}"

    # 2. Fallback Mock Data (for testing without a file)
    else:
        print(f"    ⚠️ [Tool] File not found. Using MOCK data for demonstration.")
        mock_content = """
Research Paper: Advanced AI Systems

Abstract: This paper presents a novel framework for multi-agent AI systems that can collaborate to solve complex research problems.

Introduction: The field of artificial intelligence has rapidly evolved with the emergence of large language models and agent-based architectures.

Methodology: We employed a sequential workflow combining PDF analysis, parallel processing, and result aggregation using the Google ADK framework.

Key Findings:
- Multi-agent systems show 95% improvement in research analysis speed
- Parallel execution reduces processing time by 60%
- Integration with real-time search enables up-to-date information retrieval

Conclusion: Our approach demonstrates significant improvements in automating research paper analysis.
        """
        return mock_content

print("✅ PDF Search Tool initialized.")

# 1. PDF Reader Agent 
pdf_reader_agent = Agent(
    name="PDFReader",
    model=Gemini(model=MODEL_NAME, api_key=GOOGLE_API_KEY, retry_options=retry_config),
    instruction="""You are an expert document researcher. 
    Your job is to use the `search_pdf_tool` to extract the full content from document.pdf.
    Then analyze it to provide:
    1. **Main Topic**: What is the paper about?
    2. **Key Contributions**: Novel contributions and innovations
    3. **Methodology**: Approaches and methods used
    4. **Results/Findings**: Main outcomes and conclusions
    
    Cite specific sections from the document.""",
    tools=[FunctionTool(search_pdf_tool)],
    output_key="pdf_findings"
)
print("✅ PDF Reader Agent created.")

✅ PDF Search Tool initialized.
✅ PDF Reader Agent created.


In [83]:
# 2. Summarizer Agent
summarizer_agent = Agent(
    name="Summarizer",
    model=Gemini(model=MODEL_NAME, api_key=GOOGLE_API_KEY, retry_options=retry_config),
    instruction="""You are an expert scientific paper analyst. 
    Read the research paper content provided: {pdf_findings}
    
    Create a comprehensive summary that includes:
    1. **Main Topic**: What is the paper about?
    2. **Key Contributions**: What are the novel contributions and innovations?
    3. **Methodology**: What approaches or methods were used?
    4. **Results/Findings**: What were the main outcomes?
    
    Keep the summary clear, structured, and under 200 words.
    If the findings are empty, state that no information was found.""",
    output_key="final_summary"
)

In [84]:
# 3. Tech Researcher
tech_researcher = Agent(
    name="Tech_Researcher",
    model=Gemini(model=MODEL_NAME, api_key=GOOGLE_API_KEY, retry_options=retry_config),

    instruction="""
You are a senior research analyst.

Input: {pdf_findings}

1. Extract the paper’s **main technical focus**, research problem, and method.
2. Evaluate the paper technically:
   - What is innovative?
   - What is weak or missing?
   - What assumptions does it make?
   - Possible real-world applications?
3. Perform a web search using the search tool:
   - Find the latest (2024–2025) work, breakthroughs, or criticisms related to the same topic.
   - Prefer scholarly or technical sources.
4. Produce a concise synthesis (max 100 words):
   - Technical evaluation of the paper
   - How the latest research trends compare or validate/challenge it
   - Missing gaps or future directions

Your output must be factual, technical, and short.
""",

    tools=[google_search],
    output_key="tech_research"
)


In [85]:
# The ParallelAgent runs all its sub-agents simultaneously
# other agents can be added here later
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[summarizer_agent, tech_researcher],
)

# Agregate results from parallel agents
research_aggregator = Agent(
    name="ResearchAggregator",
    model=Gemini(model=MODEL_NAME, api_key=GOOGLE_API_KEY, retry_options=retry_config),
    instruction="""
You are a research synthesis expert.
Input:
1. Summary from Summarizer Agent: {final_summary}
2. Technical research from Tech Researcher Agent: {tech_research}
Your task is to combine these inputs into a single, coherent research report that addresses the user's original question. Ensure the report is clear, concise, and well-structured.
""",
    output_key="research_report"
)

# Create the Sequential Agent to agregate read PDF, then run workflow
agents = SequentialAgent(
    name="ResearchWorkflowAgent",
    sub_agents=[pdf_reader_agent, parallel_research_team, research_aggregator],
)


In [86]:
runner = InMemoryRunner(agent=agents)
response = await runner.run_debug(
    "Analyse document.pdf the document and provide a comprehensive summary of the key findings and methodology."
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/admin/HF/Agents5D/.venv/lib/python3.10/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Analyse document.pdf the document and provide a comprehensive summary of the key findings and methodology.
    🔎 [Tool] Searching PDF 'document.pdf' for: ''
PDFReader > This paper, "Tagging Amazigh with AnCoraPipe," addresses the scarcity of Natural Language Processing (NLP) tools and resources for the Amazigh language. The authors aim to introduce morphosyntactic tagging for Amazigh, utilizing the AnCora Pipe multilevel annotation tool.

**Main Topic**: The primary focus of the paper is to present the features of the Amazigh language for morphology annotation, specifically addressing its tagging with the AnCora Pipe tool. This endeavor is viewed as an initial step towards the automatic processing of the Amazigh language, with the ultimate goal of developing a part-of-speech tagger. (Abstract, Introduction)

**Key Contributions**:
*   **Addressing Tool Scarcity**: The paper proposes to equip the Amazigh language with an important morp